<a href="https://colab.research.google.com/github/David2024patton/Notate/blob/main/Python_Installer_Script_(Dynamic_IP_Support).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import os
import secrets
import string
import subprocess
import webbrowser
from pathlib import Path
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout,
    QLabel, QLineEdit, QComboBox, QPushButton, QCheckBox,
    QScrollArea, QGridLayout, QGroupBox, QAction, QMessageBox,
    QTextEdit
)
from PyQt5.QtGui import QIcon, QFont
from PyQt5.QtCore import Qt

# === CONFIG ===
services = {
    "n8n": {"port": 5678, "image": "n8nio/n8n:latest"},
    "flowise": {"port": 3000, "image": "flowiseai/flowise"},
    "supabase": {"port": 5432, "image": "supabase/postgres"},
    "qdrant": {"port": 6333, "image": "qdrant/qdrant"},
    "zep": {"port": 5001, "image": "getzep/zep"},
    "litellm": {"port": 8080, "image": "berriai/litellm"},
    "langfuse": {"port": 4000, "image": "langfuse/langfuse"},
    "ollama": {"port": 11434, "image": "ollama/ollama"},
    "browserless": {"port": 3001, "image": "browserless/chrome"},
    "neo4j": {"port": 7687, "image": "neo4j:latest"},
    "langchain": {"port": 8501, "image": "langchainai/langchain-serve"},
    "fastapi": {"port": 8000, "image": "python:3.9-slim-buster"},
    "redis": {"port": 6379, "image": "redis:latest"},
    "openwebui": {"port": 8082, "image": "openwebui/openwebui"},
    "searxng": {"port": 8888, "image": "searxng/searxng"},
    "openrouter": {"port": 3002, "image": "openrouter/openrouter"},
    "huggingface": {"port": 8502, "image": "huggingface/transformers:latest"},
    "grog": {"port": 9200, "image": "docker.elastic.co/elasticsearch/elasticsearch:8.12.1"},
    "cohere": {"port": 7000, "image": "cohere/cohere-ai"},
    "stabilityai": {"port": 7001, "image": "stabilityai/stable-diffusion-cli"},
    "civitai": {"port": 7002, "image": "civitai/civitai"},
    "textgen": {"port": 5002, "image": "oobabooga/text-generation-webui"},
    "pinokio": {"port": 5003, "image": "pinokiodev/pinokio"},
    "drawui": {"port": 5010, "image": "lukefx/draw-a-ui"},
    "facefusion": {"port": 5011, "image": "facefusion/facefusion"},
    "stablestudio": {"port": 5012, "image": "stabilityai/stable-studio"},
    "a1111": {"port": 5013, "image": "ghcr.io/automatic1111/stable-diffusion-webui"},
    "llamacleaner": {"port": 5014, "image": "ghcr.io/napiquet/lama-cleaner"},
    "gptpilot": {"port": 5015, "image": "pythagoraio/gpt-pilot"},
    "refact": {"port": 5016, "image": "refactai/refact-self-hosted"},
    "photomaker": {"port": 5017, "image": "cppla/photomaker"},
    "comfyui": {"port": 5018, "image": "comfyanonymous/comfyui"},
    "slingring": {"port": 5019, "image": "slingring/slingring"},
    "serge": {"port": 5020, "image": "serge-ai/serge"},
    "lobechat": {"port": 5021, "image": "lobehub/lobe-chat"},
    "nextpy": {"port": 5022, "image": "reflexdev/reflex"},
    "scrapy": {"port": 5023, "image": "scrapy/scrapy"},
    "playwright": {"port": 5024, "image": "mcr.microsoft.com/playwright:v1.41.0-python3.10"},
    "beautifulsoup": {"port": 5025, "image": "python:3.9-slim-buster"}
}

DEFAULT_GPU_TYPE = "NVIDIA"
DEFAULT_CONTAINER_ENGINE = "docker"
BASE_DIR = os.path.dirname(os.path.abspath(__file__))

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("AI Stack Installer")
        self.setWindowIcon(QIcon(os.path.join(BASE_DIR, "icon.png")))
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout(self.central_widget)

        # --- General Settings ---
        self.general_group = QGroupBox("General Settings")
        self.general_layout = QGridLayout()

        self.container_name_label = QLabel("Container Name:")
        self.container_name_input = QLineEdit("ai_stack")
        self.general_layout.addWidget(self.container_name_label, 0, 0)
        self.general_layout.addWidget(self.container_name_input, 0, 1)

        self.gpu_label = QLabel("GPU Type:")
        self.gpu_combo = QComboBox()
        self.gpu_combo.addItems(["NVIDIA", "AMD", "CPU"])
        self.gpu_combo.setCurrentText(DEFAULT_GPU_TYPE)
        self.general_layout.addWidget(self.gpu_label, 1, 0)
        self.general_layout.addWidget(self.gpu_combo, 1, 1)

        self.username_label = QLabel("Admin Username:")
        self.username_input = QLineEdit()
        self.general_layout.addWidget(self.username_label, 2, 0)
        self.general_layout.addWidget(self.username_input, 2, 1)

        self.password_label = QLabel("Admin Password:")
        self.password_input = QLineEdit()
        self.password_input.setEchoMode(QLineEdit.Password)
        self.general_layout.addWidget(self.password_label, 3, 0)
        self.general_layout.addWidget(self.password_input, 3, 1)

        self.email_label = QLabel("Email (for HTTPS):")
        self.email_input = QLineEdit()
        self.general_layout.addWidget(self.email_label, 4, 0)
        self.general_layout.addWidget(self.email_input, 4, 1)

        self.engine_label = QLabel("Container Engine:")
        self.engine_combo = QComboBox()
        self.engine_combo.addItems(["Docker", "Podman"])
        self.engine_combo.setCurrentText(DEFAULT_CONTAINER_ENGINE)
        self.general_layout.addWidget(self.engine_label, 5, 0)
        self.general_layout.addWidget(self.engine_combo, 5, 1)

        self.dynamic_ip_label = QLabel("Dynamic IP from No-IP.com:")  # Add Dynamic IP Label
        self.dynamic_ip_checkbox = QCheckBox()
        self.general_layout.addWidget(self.dynamic_ip_label, 6, 0)
        self.general_layout.addWidget(self.dynamic_ip_checkbox, 6, 1)


        self.general_group.setLayout(self.general_layout)
        self.layout.addWidget(self.general_group)

        # --- Service Selection ---
        self.services_group = QGroupBox("Select Services")
        self.services_layout = QVBoxLayout()
        self.services_scroll = QScrollArea()
        self.services_scroll_widget = QWidget()
        self.services_grid_layout = QGridLayout(self.services_scroll_widget)
        self.service_checkboxes = {}
        row, col = 0, 0
        sorted_services = sorted(services.keys())
        for i, name in enumerate(sorted_services):
            checkbox = QCheckBox(f"{name.title()} (Port: {services[name]['port']})")
            self.services_grid_layout.addWidget(checkbox, row, col)
            self.service_checkboxes[name] = checkbox
            col += 1
            if col > 2:
                col = 0
                row += 1
        self.services_scroll.setWidget(self.services_scroll_widget)
        self.services_scroll.setWidgetResizable(True)
        self.services_layout.addWidget(self.services_scroll)

        self.select_all_checkbox = QCheckBox("Select All Services")
        self.select_all_checkbox.stateChanged.connect(self.toggle_select_all)
        self.services_layout.addWidget(self.select_all_checkbox)

        self.services_group.setLayout(self.services_layout)
        self.layout.addWidget(self.services_group)

        # --- Theme and Actions ---
        self.theme_button = QPushButton("Toggle Dark Theme")
        self.theme_button.clicked.connect(self.toggle_theme)
        self.layout.addWidget(self.theme_button)

        self.deploy_button = QPushButton("Generate and Deploy AI Stack")
        self.deploy_button.setDefault(True)
        self.deploy_button.clicked.connect(self.start_deployment)
        self.layout.addWidget(self.deploy_button)

        self.output_group = QGroupBox("Deployment Output")
        self.output_layout = QVBoxLayout()
        self.output_text = QTextEdit()
        self.output_text.setReadOnly(True)
        self.output_layout.addWidget(self.output_text)
        self.output_group.setLayout(self.output_layout)
        self.layout.addWidget(self.output_group)

        # --- Theme Handling ---
        self.dark_mode = False
        self.default_stylesheet = QApplication.instance().styleSheet()
        self.dark_stylesheet = """
            QMainWindow { background-color: #36393F; color: #DCDCDC; }
            QGroupBox { color: #DCDCDC; border: 1px solid #555; margin-top: 2ex; }
            QGroupBox::title { subcontrol-origin: margin; subcontrol-position: top left; padding: 0 3px; color: #DCDCDC; }
            QLineEdit { background-color: #444; color: #EEEEEE; border: 1px solid #555; }
            QComboBox { background-color: #444; color: #EEEEEE; border: 1px solid #555; }
            QCheckBox { color: #EEEEEE; }
            QPushButton { background-color: #555; color: #EEEEEE; border: 1px solid #777; padding: 5px; }
            QPushButton:hover { background-color: #777; }
            QTextEdit { background-color: #444; color: #EEEEEE; border: 1px solid #555; }
        """

        # --- Load/Save Config ---
        menubar = self.menuBar()
        file_menu = menubar.addMenu("File")
        save_action = QAction("Save Configuration", self)
        save_action.triggered.connect(self.save_config)
        file_menu.addAction(save_action)
        load_action = QAction("Load Configuration", self)
        load_action.triggered.connect(self.load_config)
        file_menu.addAction(load_action)

        self.load_config()

    def toggle_select_all(self, state):
        if state == Qt.Checked:
            for checkbox in self.service_checkboxes.values():
                checkbox.setChecked(True)
        else:
            for checkbox in self.service_checkboxes.values():
                checkbox.setChecked(False)

    def toggle_theme(self):
        self.dark_mode = not self.dark_mode
        if self.dark_mode:
            app.setStyleSheet(self.dark_stylesheet)
        else:
            app.setStyleSheet(self.default_stylesheet)

    def save_config(self):
        config = {
            "container_name": self.container_name_input.text(),
            "gpu_type": self.gpu_combo.currentText(),
            "username": self.username_input.text(),
            "password": self.password_input.text(),
            "email": self.email_input.text(),
            "engine": self.engine_combo.currentText(),
            "selected_services": [name for name, checkbox in self.service_checkboxes.items() if checkbox.isChecked()],
            "select_all": self.select_all_checkbox.isChecked(),
            "dynamic_ip": self.dynamic_ip_checkbox.isChecked() # Save dynamic ip setting
        }
        config_path = os.path.join(BASE_DIR, "config.json")
        try:
            import json
            with open(config_path, 'w') as f:
                json.dump(config, f, indent=4)
            QMessageBox.information(self, "Configuration", f"Configuration saved to {config_path}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Failed to save configuration: {e}")

    def load_config(self):
        config_path = os.path.join(BASE_DIR, "config.json")
        try:
            import json
            if os.path.exists(config_path):
                with open(config_path, 'r') as f:
                    config = json.load(f)
                self.container_name_input.setText(config.get("container_name", "ai_stack"))
                self.gpu_combo.setCurrentText(config.get("gpu_type", DEFAULT_GPU_TYPE))
                self.username_input.setText(config.get("username", ""))
                self.password_input.setText(config.get("password", ""))
                self.email_input.setText(config.get("email", ""))
                self.engine_combo.setCurrentText(config.get("engine", DEFAULT_CONTAINER_ENGINE))
                selected_services = config.get("selected_services", [])
                for name, checkbox in self.service_checkboxes.items():
                    checkbox.setChecked(name in selected_services)
                self.select_all_checkbox.setChecked(config.get("select_all", False))
                self.dynamic_ip_checkbox.setChecked(config.get("dynamic_ip", False)) # Load dynamic ip setting
                QMessageBox.information(self, "Configuration", f"Configuration loaded from {config_path}")
            else:
                QMessageBox.information(self, "Configuration", "No existing configuration found. Starting with defaults.")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Failed to load configuration: {e}")

    def generate_secret(self, length=32):
        alphabet = string.ascii_letters + string.digits
        return ''.join(secrets.choice(alphabet) for _ in range(length))

    def create_env_file(self, container_name, username, password, email):
        env_lines = [
            f"CONTAINER_NAME={container_name}",
            f"ADMIN_USERNAME={username}",
            f"ADMIN_PASSWORD={password}",
            f"EMAIL={email}",
            f"SECRET_KEY={self.generate_secret()}",
            f"JWT_SECRET={self.generate_secret()}",
            f"DB_PASSWORD={self.generate_secret(16)}",
        ]
        env_file_path = os.path.join(BASE_DIR, ".env")
        Path(env_file_path).write_text("\n".join(env_lines))
        self.output_text.append("✅ .env file created.")

    def create_compose_file(self, container_name, gpu_type, selected_services, dynamic_ip): # Add dynamic_ip
        template_path = os.path.join(BASE_DIR, "configs", "docker-compose-template.yml")
        try:
            with open(template_path, 'r') as f:
                compose_content = f.read()
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Failed to read docker-compose template: {e}")
            return None

        services_yml = []
        for name in selected_services:
            service_config = services[name]
            port = service_config["port"]
            image = service_config["image"]
            service_name_in_compose = name.replace(" ", "")
            volumes = []
            if name == "n8n":
                volumes.append(f"./services/n8n/backup/workflows:/root/.n8n/workflows")
            # Add environment variable for dynamic IP
            extra_env = []
            if dynamic_ip and name in ["n8n"]:
                extra_env.append("NODE_URL=http://host.docker.internal:5678")

            services_yml.append(f"""
  {service_name_in_compose}:
    image: {image}
    container_name: {container_name}_{service_name_in_compose}
    restart: unless-stopped
    environment:
      - VIRTUAL_HOST={name}.localhost
      - VIRTUAL_PORT={port}
      - LETSENCRYPT_HOST={name}.localhost
      - LETSENCRYPT_EMAIL=${{EMAIL}}
      {chr(10).join(f"      - {e}" for e in extra_env)}
    ports:
      - "{port}:{port}"
    volumes:
      {chr(10).join(f"      - {v}" for v in volumes)}
    networks:
      - ai_net
""")

        gpu_args = 'runtime: nvidia' if gpu_type == "NVIDIA" else ''
        insertion_point = "  # --- SERVICES WILL BE ADDED HERE BY THE PYTHON SCRIPT ---"
        compose_content = compose_content.replace(insertion_point, ''.join(services_yml))

        if gpu_type == "NVIDIA":
            compose_content = compose_content.replace("services:", f"services:\n  nginx-proxy:\n    {gpu_args}")

        compose_file_path = os.path.join(BASE_DIR, "docker-compose.yml")
        Path(compose_file_path).write_text(compose_content)
        self.output_text.append("✅ docker-compose.yml created.")
        return compose_file_path

    def check_podman_prereqs(self):
        self.output_text.append("\n⚠️ Podman selected. Ensure you have the following installed:")
        self.output_text.append("- podman")
        self.output_text.append("- podman-compose")
        self.output_text.append("- systemd or WSL2 support")

    def deploy_stack(self, engine, compose_file):
        if engine == "docker":
            cmd = ["docker", "compose", "-f", compose_file, "up", "-d"]
        else:
            cmd = ["podman-compose", "-f", compose_file, "up", "-d"]
        self.output_text.append(f"🚀 Deploying stack using: {' '.join(cmd)}")
        try:
            subprocess.run(cmd, shell=False, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            self.output_text.append("✅ Stack deployed successfully!")
        except subprocess.CalledProcessError as e:
            self.output_text.append(f"❌ Deployment failed: {e}")
            self.output_text.append(f"  Output: {e.output.decode()}")
            QMessageBox.critical(self, "Deployment Error",
                                 f"Failed to deploy stack. Check the output log for errors. Error:{e}")

    def open_services(self, selected_services):
        self.output_text.append("\n🌐 Opening service URLs...\n")
        for name in selected_services:
            service_name = name.replace(" ", "")
            url = f"https://{name}.localhost"
            self.output_text.append(f"{name.title()} => {url}")
            webbrowser.open(url)

    def start_deployment(self):
        container_name = self.container_name_input.text()
        gpu_type = self.gpu_combo.currentText()
        username = self.username_input.text()
        password = self.password_input.text()
        email = self.email_input.text()
        engine = self.engine_combo.currentText().lower()
        dynamic_ip = self.dynamic_ip_checkbox.isChecked() # Get dynamic IP setting
        if self.select_all_checkbox.isChecked():
            selected_services = list(services.keys())
        else:
            selected_services = [name for name, checkbox in self.service_checkboxes.items() if checkbox.isChecked()]

        if not selected_services:
            QMessageBox.warning(self, "Warning", "Please select at least one service to install.")
            return

        self.output_text.clear()
        self.output_text.append("Starting Deployment Process...\n")
        self.output_text.append(f"Container Name: {container_name}")
        self.output_text.append(f"GPU Type: {gpu_type}")
        self.output_text.append(f"Container Engine: {engine}")
        self.output_text.append(f"Dynamic IP: {dynamic_ip}")
        self.output_text.append(f"Selected Services: {', '.join(selected_services)}")

        if engine == "podman":
            self.check_podman_prereqs()

        self.create_env_file(container_name, username, password, email)
        compose_file = self.create_compose_file(container_name, gpu_type, selected_services, dynamic_ip) # Pass dynamic_ip
        if compose_file:
            self.deploy_stack(engine, compose_file)
            self.open_services(selected_services)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_window = MainWindow()
    main_window.show()
    sys.exit(app.exec_())